# Module Imports

In [1]:
## IMPORT MODULES
import pandas as pd
import numpy as np
from sklearn import linear_model


In [2]:
## IMPORT PLOTTING MODULES
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import matplotlib as mpl
from matplotlib import cm
%matplotlib widget

# Dataframe

In [3]:
## FULL DATAFRAME
df_raw = pd.read_csv("XNS_2-Policy_Summary.csv")

## Drop extraneous data points
df_raw = df_raw.drop(df_raw[
    (df_raw['landing_rate']<=0.1) & 
    (df_raw['vz_d']>=2.75)
    ].index)

df_raw = df_raw.drop(df_raw[
    (df_raw['landing_rate']<=0.1) & 
    (df_raw['vx_d']>=1.0)
    ].index)

df_raw_avg = df_raw.groupby(['vz_d','vx_d']).agg([np.mean,np.std]).reset_index()


# df_temp = df.query(f"landing_rate>={0.2}")
# df_temp_avg = df_raw.groupby(['vz_d','vx_d']).agg([np.mean,np.std]).reset_index()


df = df_raw
df['My_d'] = df['My_d'].apply(lambda x: np.abs(x)) # Convert My to magnitude
# df['My_d'] = df['My_d'].apply(lambda x: 7.7 if x>7.7 else x) # Cap My_d to max moment
df = df.drop(df[(df['vz_d']<= 2.25) & (df['vx_d']<= 0.75)].index) # Drop corner with no successful landings
df = df.dropna()

## AVERAGED DATAFRAME
df_avg = df.groupby(['vz_d','vx_d']).agg([np.mean,np.std]).reset_index()

In [4]:
df

,vz_d,vx_d,trial_num,landing_rate,RREV_threshold,G1,RREV_sig,G1_sig,RREV_trigger,OF_y,My_d,impact_eul,impact_tdelta,alpha_mu,alpha_sigma,mu_ini,sigma_ini
35,2.25,1.00,0,0.619048,4.56,15.55,0.00,0.59,5.148077,-2.085769,15.500846,-130.195882,0.192154,[0. 0.],[0. 0.],[3.04 6.46],[1.5 1.5]
36,2.25,1.00,1,0.500000,4.79,7.18,0.00,0.05,5.118778,-2.072111,7.192000,-150.963279,0.198444,[0. 0.],[0. 0.],[2.6 6.34],[1.5 1.5]
37,2.25,1.00,2,0.526316,4.37,4.41,0.05,0.02,4.898600,-1.984100,4.417100,-157.195354,0.245600,[0. 0.],[0. 0.],[1.15 4.76],[1.5 1.5]
40,2.25,1.00,5,0.411765,4.32,5.67,0.02,0.00,4.966000,-2.012000,5.671429,-158.653029,0.213714,[0. 0.],[0. 0.],[5.24 5.53],[1.5 1.5]
44,2.25,1.25,3,0.545455,4.46,9.75,0.00,0.31,5.047667,-2.554667,9.581000,-128.006651,0.191333,[0. 0.],[0. 0.],[5.81 4.28],[1.5 1.5]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,4.00,2.50,3,1.000000,2.80,11.72,0.00,0.86,3.045000,-1.811000,11.673000,-125.230059,0.240000,[0. 0.],[0. 0.],[1.98 5.83],[1.5 1.5]
573,4.00,2.50,4,0.681818,4.59,5.72,0.00,0.00,4.924200,-2.919933,5.718067,-51.719035,0.235867,[0. 0.],[0. 0.],[5.93 5.89],[1.5 1.5]
574,4.00,2.50,5,0.631579,3.74,1.66,0.00,0.00,4.022250,-2.386917,1.658917,-148.378890,0.354500,[0. 0.],[0. 0.],[4.44 1.78],[1.5 1.5]
576,4.00,2.75,1,0.500000,4.86,6.88,0.00,0.03,5.102667,-3.343000,6.878667,-166.136087,0.214000,[0. 0.],[0. 0.],[2.54 4.52],[1.5 1.5]


# Landing Rate Data

In [5]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")
ax.scatter(df['vx_d'],df['vz_d'],df['landing_rate'])

ax.set_zlim(0,1)

ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('Landing Rate')
ax.set_title('Landing Rate (Raw Data)')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
## AVG LANDING RATE SURFACE
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")
cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=0,vmax=1)
pcm = ax.plot_trisurf(df_raw_avg['vx_d'],df_raw_avg['vz_d'],df_raw_avg['landing_rate','mean'],cmap = cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing %")
df
ax.set_zlim(0,1)

ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('Landing Rate')
ax.set_title('Avg Landing Rate for Final 3 Episodes')


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# RREV vs IC

In [7]:
## RREV Threshold vs IC
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

ax.scatter(df['vx_d'],df['vz_d'],df['RREV_threshold'])


ax.set_xlim(0,2.75)
ax.set_ylim(1.0,4.5)


ax.set_xlabel('Vx_d')
ax.set_ylabel('Vz_d')
ax.set_zlabel('RREV_threshold')
ax.set_title('RREV_thr vs IC - (Raw Data)')


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:

## Avg RREV_thr vs IC
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=4,vmax=5)
pcm = ax.plot_trisurf(df_avg['vx_d'],df_avg['vz_d'],df_avg['RREV_threshold','mean'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="RREV_threshold")

ax.set_xlim(0,2.75)
ax.set_ylim(1.0,4.5)
ax.set_zlim(3,6)

ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('RREV_threshold')

ax.set_title('Avg RREV_thr vs IC')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Rotation Time Data

In [9]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

df_temp = df.query(f"landing_rate>={0.1}")

cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=4,vmax=5)
pcm = ax.scatter(df_temp['vx_d'],df_temp['vz_d'],df_temp['impact_tdelta'])
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="RREV_threshold")


ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('Delta_t [s]')

ax.set_title('Time Rotating vs IC (Raw Data)')

ax.set_zlim(0,0.5)




plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

df_temp = df.query(f"landing_rate>={0.1}")

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=4)

pcm = ax.scatter(df_temp['RREV_trigger'],df_temp['landing_rate'],df_temp['impact_tdelta'],c=df_temp['vz_d'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Vz_d')


ax.set_xlabel('RREV_trigger')
ax.set_ylabel('Landing_rate')
ax.set_zlabel('Delta_t [s]')

ax.set_title('Time Rotating vs RREV (Raw Data)')

ax.set_zlim(0,0.5)




plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# My_d vs IC Data

In [11]:
## Define Dataframe

df_temp = df.query(f"landing_rate>={0.1}")


fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

pcm = ax.scatter(df['vx_d'],df['vz_d'],df['My_d'])



ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('My_d [N*mm]')

ax.set_title('My_d vs IC (Raw Data)')

## I have more data for vz = (1.5,2.75) than vz = (3.00,4.0) so My_d differences are likely not a trend, just sparse data




plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Impact Angle Data

In [12]:
df_temp = df.query(f"impact_eul<={-60}")
# df_temp_avg = df_raw.groupby(['vz_d','vx_d']).agg([np.mean,np.std]).reset_index()



## AVG LANDING RATE PLOT
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
pcm = ax.scatter(df_temp['vx_d'],df_temp['vz_d'],-df_temp['impact_eul'],c=df_temp['landing_rate'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing %')

ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('Impact Angle')


ax.set_zlim(60,180)
ax.set_zticks([60,90,120,150,180])

## Failed landings in corner
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
## CLEAN DATAFRAME
df_temp = df.query(f"impact_eul<={-60}")
df_temp_avg = df_temp.groupby(['vz_d','vx_d']).agg([np.mean,np.std]).reset_index()


## AVG LANDING RATE PLOT
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=0,vmax=180)
ax.plot_trisurf(df_temp_avg['vx_d'],df_temp_avg['vz_d'],-df_temp_avg['impact_eul','mean'],cmap=cmap)


ax.set_xlim(0,2.75)
ax.set_ylim(1.0,4.0)
ax.set_zlim(60,180)
ax.set_zticks([60,90,120,150,180])


ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('Impact Angle')

ax.set_title('Avg Impact Angle vs IC')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Moment vs RREV

In [14]:
## CLEAN DATAFRAME
df_temp = df.query(f"My_d<={7.7}")
df_temp_avg = df_temp.groupby(['vz_d','vx_d']).agg([np.mean,np.std]).reset_index()


## AVG LANDING RATE PLOT
fig = plt.figure()
ax = fig.add_subplot(111)


cmap = mpl.cm.Greys
norm = mpl.colors.Normalize(vmin=0,vmax=1)

ax.scatter(df_temp['RREV_threshold'],df_temp['My_d'],c=df_temp['landing_rate'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing %')

ax.grid()
ax.set_xlim(2,7)
ax.set_ylim(0,10)
ax.hlines(7.77,2,7)
ax.text(2.05,8.0,'Motors Maxed Out')

ax.set_xlabel('RREV_thr')
ax.set_ylabel('My_d [N*mm]')
ax.set_title('Policy: My_d vs RREV_thr')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
## CLEAN DATAFRAME
df_temp = df.query(f"My_d<={7.7}")
df_temp_avg = df_temp.groupby(['vz_d','vx_d']).agg([np.mean,np.std]).reset_index()

## AVG LANDING RATE PLOT
fig = plt.figure()
ax = fig.add_subplot(111)


cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=0,vmax=3)

ax.scatter(df_temp['RREV_threshold'],df_temp['My_d'],c=df_temp['vz_d'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Vz_d')

ax.grid()
ax.set_xlim(2,7)
ax.set_ylim(0,10)
ax.hlines(7.77,2,7)
ax.text(2.05,8.0,'Motors Maxed Out')

ax.set_xlabel('RREV_thr')
ax.set_ylabel('My_d [N*mm]')
ax.set_title('Policy: My_d vs RREV_thr')
## As Vz increases RREV_trigger decreases to preserve time to contact. It also finds a host of My_d that work and they vary because of the robustness of the legs and the fact that landing success isn't deterministic.
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
## CLEAN DATAFRAME
df_temp = df.query(f"My_d<={7.7}")
df_temp_avg = df_temp.groupby(['vz_d','vx_d']).agg([np.mean,np.std]).reset_index()

## AVG LANDING RATE PLOT
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)

ax.scatter(df_temp['RREV_threshold'],df_temp['vx_d'],df_temp['My_d'],c=df_temp['landing_rate'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing %')

ax.set_zlim(0,10)
ax.set_xlim(2,7)

ax.set_xlabel('RREV_trigger')
ax.set_ylabel('vx_d')
ax.set_zlabel('My_d')


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# POLICY FITTING

## RAW POLICY RELATION

In [19]:
## CLEAN DATAFRAME
df_temp = df.query(f"My_d<={7.7}") # Remove extraneous moments higher than max

## PLOT DATA
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)

ax.scatter(df_temp['RREV_threshold'],df_temp['OF_y'],df_temp['My_d'],c=df_temp['landing_rate'],cmap=cmap,norm=norm)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing Rate')

ax.set_xlim(6,2)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)

ax.set_xlabel('RREV')
ax.set_ylabel('OF_y')
ax.set_zlabel('My_d')

ax.set_title('Raw Policy Relation')


plt.show()

# Note: the planar look to the data

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
## CLEAN DATAFRAME
df_temp = df.query(f"My_d<={7.7}") # Remove extraneous moments higher than max
df_temp = df_temp.query(f"RREV_threshold>{3.5}") # Remove outlier data
df_temp = df_temp.query(f"landing_rate>={0.8}")

## PLOT DATA
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)

ax.scatter(df_temp['RREV_threshold'],df_temp['OF_y'],df_temp['My_d'],c=df_temp['landing_rate'],cmap=cmap,norm=norm)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing Rate')

ax.set_xlim(6,2)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)

ax.set_xlabel('RREV')
ax.set_ylabel('OF_y')
ax.set_zlabel('My_d')

ax.set_title('Raw Policy Relation (LR > 80%)')


plt.show()

# Note: the planar look to the data

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
## CLEAN DATAFRAME
df_temp = df.query(f"My_d<={7.7}") # Remove extraneous moments higher than max
df_temp = df_temp.query(f"RREV_threshold>{3.5}") # Remove outlier data
df_temp = df_temp.query(f"landing_rate>={0.5}")

## PLOT DATA
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)

ax.scatter(df_temp['RREV_threshold'],df_temp['OF_y'],df_temp['My_d'],c=df_temp['landing_rate'],cmap=cmap,norm=norm)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing Rate')

ax.set_xlim(6,2)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)

ax.set_xlabel('RREV')
ax.set_ylabel('OF_y')
ax.set_zlabel('My_d')

ax.set_title('Raw Policy Relation (LR > 50%)')


plt.show()

# Note: the planar look to the data

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# LINEAR MODEL FIT

## Regression with y-based loss

In [22]:
## CLEAN DATAFRAME
df_clean = df.query(f"My_d<={7.7}") # Remove extraneous moments higher than max
df_clean = df_clean.query(f"RREV_threshold>{3.5}") # Remove outlier data
df_reg1 = df_clean.query(f"landing_rate>={0.6}")


X = df_reg1[['RREV_threshold','OF_y']]
Y = df_reg1['My_d']

reg = linear_model.LinearRegression(normalize=True)
reg.fit(X,Y)
intercept = reg.intercept_
coeffs = reg.coef_

print(f"Equation: My_d = {intercept:.2f} + {coeffs[0]:.2f}*RREV + {coeffs[1]:.2f}*OF_y \n")


## PLOT DATA
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")


My_reg = intercept + coeffs[0]*df_reg1['RREV_threshold'] + coeffs[1]*df_reg1['OF_y']
ax.plot_trisurf(df_reg1['RREV_threshold'],df_reg1['OF_y'],My_reg,label='Linear_Regression',color='blue',zorder=2)


cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing Rate')

ax.scatter(df_reg1['RREV_threshold'],df_reg1['OF_y'],df_reg1['My_d'],c=df_reg1['landing_rate'],cmap=cmap,norm=norm,zorder=1)



ax.set_xlim(6,2)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)


ax.set_xlabel('RREV')
ax.set_ylabel('OF_y')
ax.set_zlabel('My_d')


plt.show()

## Regression Analysis

import statsmodels.api as ssm       # for detail description of linear coefficients, intercepts, deviations, and many more
X=ssm.add_constant(X)               # to add constant value in the model
model= ssm.OLS(Y,X).fit()           # fitting the model
predictions= model.summary()        # summary of the model
predictions





Equation: My_d = -14.41 + 4.30*RREV + 0.03*OF_y 



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   My_d   R-squared:                       0.646
Model:                            OLS   Adj. R-squared:                  0.638
Method:                 Least Squares   F-statistic:                     84.85
Date:                Sat, 06 Feb 2021   Prob (F-statistic):           1.07e-21
Time:                        19:22:37   Log-Likelihood:                -120.32
No. Observations:                  96   AIC:                             246.6
Df Residuals:                      93   BIC:                             254.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const            -14.4138      1.463     -9.850      0.000     -17.320     -11.508
RREV_threshold     4.2972      0.336     12.775      0.000       3.629       4.965
OF_y               0.0280      0.074      0.380      0.705      -0.118       0.174
==============================================================================
Omnibus:                        3.289   Durbin-Watson:                   2.222
Prob(Omnibus):                  0.193   Jarque-Bera (JB):                3.433
Skew:                           0.067   Prob(JB):                        0.180
Kurtosis:                       3.917   Cond. No.                         90.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [23]:
# def r_squared(y, y_hat): #Manual R^2 value
#     y_bar = y.mean()
#     ss_tot = ((y-y_bar)**2).sum()
#     ss_res = ((y-y_hat)**2).sum()
#     return 1 - (ss_res/ss_tot)

# R2 = r_squared(df_reg['My_d'],My_reg)
# print(f"R^2: {R2:.3f}")

## Regression with x-based loss


In [24]:
## CLEAN DATAFRAME
df_clean = df.query(f"My_d<={7.7}") # Remove extraneous moments higher than max
df_clean = df_clean.query(f"RREV_threshold>{3.5}") # Remove outlier data
df_reg1 = df_clean.query(f"landing_rate>={0.6}")


X = df_reg1[['My_d','OF_y']]
Y = df_reg1['RREV_threshold']

reg = linear_model.LinearRegression(normalize=True)
reg.fit(X,Y)
intercept = reg.intercept_
coeffs = reg.coef_

print(f"Equation: RREV_reg = {intercept:.2f} + {coeffs[0]:.2f}*My_d + {coeffs[1]:.2f}*OF_y \n")


## PLOT DATA
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")


RREV_reg = intercept + coeffs[0]*df_reg1['My_d'] + coeffs[1]*df_reg1['OF_y']
ax.plot_trisurf(RREV_reg,df_reg1['OF_y'],df_reg1['My_d'],label='Linear_Regression',color='blue',zorder=2)


cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing Rate')

ax.scatter(df_reg1['RREV_threshold'],df_reg1['OF_y'],df_reg1['My_d'],c=df_reg1['landing_rate'],cmap=cmap,norm=norm,zorder=1)



ax.set_xlim(6,2)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)


ax.set_xlabel('RREV')
ax.set_ylabel('OF_y')
ax.set_zlabel('My_d')


plt.show()

## Regression Analysis

import statsmodels.api as ssm       # for detail description of linear coefficients, intercepts, deviations, and many more
X=ssm.add_constant(X)               # to add constant value in the model
model= ssm.OLS(Y,X).fit()           # fitting the model
predictions= model.summary()        # summary of the model
predictions





Equation: RREV_reg = 3.70 + 0.15*My_d + -0.02*OF_y 



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         RREV_threshold   R-squared:                       0.655
Model:                            OLS   Adj. R-squared:                  0.648
Method:                 Least Squares   F-statistic:                     88.37
Date:                Sat, 06 Feb 2021   Prob (F-statistic):           3.13e-22
Time:                        19:22:38   Log-Likelihood:                 41.293
No. Observations:                  96   AIC:                            -76.59
Df Residuals:                      93   BIC:                            -68.89
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.6997      0.061     60.452      0.000       3.578       3.821
My_d           0.1482      0.012     12.775      0.000       0.125       0.171
OF_y          -0.0220      0.014     -1.626      0.107      -0.049       0.005
==============================================================================
Omnibus:                        2.225   Durbin-Watson:                   2.042
Prob(Omnibus):                  0.329   Jarque-Bera (JB):                1.836
Skew:                           0.053   Prob(JB):                        0.399
Kurtosis:                       3.669   Cond. No.                         21.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [25]:
## PLOT FULL DATA
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")


ax.plot_trisurf(RREV_reg,df_reg1['OF_y'],df_reg1['My_d'],label='Linear_Regression',color='blue',zorder=2)


cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing Rate')

ax.scatter(df_clean['RREV_threshold'],df_clean['OF_y'],df_clean['My_d'],c=df_clean['landing_rate'],cmap=cmap,norm=norm,zorder=1)



ax.set_xlim(6,2)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)


ax.set_xlabel('RREV')
ax.set_ylabel('OF_y')
ax.set_zlabel('My_d')


plt.show()



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
## CLEAN DATAFRAME
df_clean = df.query(f"My_d<={7.7}") # Remove extraneous moments higher than max
df_clean = df_clean.query(f"RREV_threshold>{3.5}") # Remove outlier data
df_reg1 = df_clean.query(f"landing_rate>={0.8}")


fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")


for ii in np.arange(1,0.2,-.2):
    df_reg1 = df.query(f"landing_rate<={ii} & landing_rate>{ii-.2} & RREV_threshold>{3.5}")
    
    X = df_reg1[['My_d','OF_y']]
    Y = df_reg1['RREV_threshold']

    reg2 = linear_model.LinearRegression(normalize=True)
    reg2.fit(X,Y)

    intercept = reg2.intercept_
    coeffs = reg2.coef_

    # print(intercept)
    # print(coeffs)

    RREV_reg = intercept + coeffs[0]*df_reg1['My_d'] + coeffs[1]*df_reg1['OF_y']
    ax.plot_trisurf(RREV_reg,df_reg1['OF_y'],df_reg1['My_d'],label='Linear_Regression')

    R2 = r_squared(df_reg1['RREV_threshold'],RREV_reg)
    print(f"R^2: {R2:.3f}")

ax.set_xlim(6,3.5)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)


ax.set_xlabel('RREV')
ax.set_ylabel('OF_y')
ax.set_zlabel('My_d')


plt.show()



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'r_squared' is not defined

array([1. , 0.8, 0.6, 0.4])

In [ ]:
import scipy.odr

## CLEAN DATAFRAME
df_clean = df.query(f"My_d<={7.7}") # Remove extraneous moments higher than max
df_clean = df_clean.query(f"RREV_threshold>{3.5}") # Remove outlier data
df_reg1 = df_clean.query(f"landing_rate>={0.8}")


# X = df_reg1[['My_d','OF_y']]
x1 = df_reg1['RREV_threshold'].to_numpy()
x2 = df_reg1['OF_y'].to_numpy()
X = np.row_stack( (x1, x2) )

Y = df_reg1['My_d'].to_numpy()

def linfit(beta,x):
    return beta[0]*x[0] + beta[1]*x[1] + beta[2]

linmod = scipy.odr.Model(linfit)
data = scipy.odr.Data(X,Y)
odrfit = scipy.odr.ODR(data,linmod,beta0 = [-0.1,0.1,4.0])
odrres = odrfit.run()
odrres.pprint()




## PLOT DATA
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")


My_reg = odrres.beta[0]*df_reg1['RREV_threshold'] + odrres.beta[1]*df_reg1['OF_y'] + odrres.beta[2]
ax.plot_trisurf(df_reg1['RREV_threshold'],df_reg1['OF_y'],My_reg,label='Linear_Regression',color='blue',zorder=2)


cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing Rate')

ax.scatter(df_reg1['RREV_threshold'],df_reg1['OF_y'],df_reg1['My_d'],c=df_reg1['landing_rate'],cmap=cmap,norm=norm,zorder=1)



ax.set_xlim(6,2)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)


ax.set_xlabel('RREV')
ax.set_ylabel('OF_y')
ax.set_zlabel('My_d')


plt.show()
